In [11]:
import logging
from typing import Callable, Literal, Optional, Union

import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.optimize import minimize

from bwlogger import StyleAdapter
import argparse
import logging
from pathlib import Path
from typing import Literal, Optional, Union

import numpy as np
import pandas as pd

from bwbbgdl import GoGet
from bwlogger import StyleAdapter, basic_setup
from bwutils import TODAY, Date

from utils import calculate_weights
from plot import plot_results

In [3]:
list({1: 2})

[1]

In [13]:
# these are in ER in USD
EM_CDS_TRACKER_DICT = {
    "BRL": "GSCDBRBE Index",
    "CNY": "GSCDCHBE Index",
    "MXN": "GSCDMEBE Index",
    "ZAR": "GSCDSOBE Index",
}

# these are in LOC ER with pnl converted to USD
IRS_TRACKER_DICT = {
    "BRL": "GSSWBRN5 Index",
    "CNY": "GSSWCNN5 Index",
    "MXN": "GSSWMXN5 Index",
    "ZAR": "GSSWZAN5 Index",
}

EQ_TRACKER_DICT = {
    "BRL": "BNPIFBR Index",  # in BRL
    "CNY": "BNPIFCNO Index",  # China onshore but with pnl converted to USD
    "ZAR": "BNPIFSA Index",  # in ZAR
    # "MXN": "???? Index",
}

# these are in ER in USD
FX_TRACKER_DICT = {x: f"JPFCT{x} Index" for x in EM_CDS_TRACKER_DICT}

# def country_assets_portfolio(
#     ccy: str,
#     vol_target: float = 0.1,
#     dt_ini: Date = "1990-12-31",
#     dt_end: Date = TODAY,
# ) -> pd.DataFrame:
mapper_ticker = FX_TRACKER_DICT
vol_target: float = 0.1
dt_ini: Date = "1990-12-31"
dt_end: Date = TODAY
method_weights = "ERC"


inverse_mapper_ticker = {v: k for k, v in mapper_ticker.items()}
tickers = list(mapper_ticker.values())

g = GoGet(enforce_strict_matching=True)

tracker_df: pd.DataFrame = g.fetch(
    tickers=tickers,
    fields="PX_LAST",
    dt_ini=dt_ini,
    dt_end=dt_end,
)

tracker_df = tracker_df.pivot_table(index="date", columns="id")
tracker_df.columns = tracker_df.columns.droplevel(0)
tracker_df = tracker_df.rename(columns=inverse_mapper_ticker).dropna()

backtest = pd.Series(index=tracker_df.index)
backtest.iloc[0] = 100.0

cov = np.log(tracker_df).diff(21).cov() * 12.0
vols = pd.Series(index=cov.index, data=np.diag(cov))
w = calculate_weights(method=method_weights, cov_matrix=cov)
adj_factor = vol_target / np.sqrt(w @ cov @ w)
w = adj_factor * w

q = backtest.iloc[0] * w / tracker_df.iloc[0]

for t, tm1 in zip(backtest.index[1:], backtest.index[:-1]):
    pnl = ((tracker_df.loc[t] - tracker_df.loc[tm1]) @ q).sum()
    backtest[t] = backtest[tm1] + pnl
    if t.month != tm1.month:
        if tracker_df.loc[:t].shape[0] > 252:
            cov = np.log(tracker_df.loc[:tm1]).diff(21).cov() * 12.0
            vols = pd.Series(index=cov.index, data=np.diag(cov))

        w = (1 / vols) / (1 / vols).sum()
        adj_factor = vol_target / np.sqrt(w @ cov @ w)
        w = adj_factor * w
        q = backtest[tm1] * w / tracker_df.loc[tm1]

backtest = pd.concat(
    [tracker_df, backtest.to_frame("assets")],
    axis=1,
    join="outer",
    sort=True,
)
backtest
# return backtest

C:\Users\pcampos\AppData\Local\Temp\ipykernel_31140\2122055859.py:56: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  backtest = pd.Series(index=tracker_df.index)


,BRL,CNY,MXN,ZAR,assets
date,,,,,
2022-08-15,279.780,116.697,190.237,152.848,100.000000
2022-08-16,276.633,116.111,189.385,152.640,99.031527
2022-08-17,275.770,116.615,188.250,150.479,98.600322
2022-08-18,275.690,116.212,188.510,149.520,98.193783
2022-08-19,273.947,115.845,187.162,147.546,97.103324
...,...,...,...,...,...
2024-07-01,285.467,108.135,231.751,146.083,95.350342
2024-07-02,281.785,108.080,234.234,143.238,94.677477
2024-07-03,286.142,108.045,235.295,145.401,95.633160


In [6]:
tracker_df

""
